In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime, timedelta

## Funções de Apoio

In [29]:
def load_data() -> pd.DataFrame:
    """ Carrega os dados necessários para o projeto"""

    # Carrega os arquivos CSV e concatenas eles em um unico dataframe
    df = pd.read_csv('../assets/dados_violencia_mulheres_ses_2022.csv', sep=';')
    df = pd.concat([df, pd.read_csv('../assets/dados_violencia_mulheres_ses_2023.csv', sep=';')])
    df = pd.concat([df, pd.read_csv('../assets/dados_violencia_mulheres_ses_2021.csv', sep=';')])

    # Converte as colunas para os tipos corretos
    df.DT_NOTIFIC = pd.to_datetime(df.DT_NOTIFIC, format='%d/%m/%Y')
    df.DT_NASC = pd.to_datetime(df.DT_NASC, format='%d/%m/%Y')
    df.replace({'NU_IDADE_N': {np.nan: -1}}, inplace=True)
    df.NU_IDADE_N = df.NU_IDADE_N.astype(int)
    df.OUT_VEZES = df.OUT_VEZES.astype('category')
    df.LES_AUTOP = df.LES_AUTOP.astype('category')
    df.VIOL_FISIC = df.VIOL_FISIC.astype('category')
    df.VIOL_PSICO = df.VIOL_PSICO.astype('category')
    df.VIOL_SEXU = df.VIOL_SEXU.astype('category')
    df.NUM_ENVOLV = df.NUM_ENVOLV.astype('category')
    df.AUTOR_SEXO = df.AUTOR_SEXO.astype('category')
    df.ORIENT_SEX = df.ORIENT_SEX.astype('category')
    df.IDENT_GEN = df.IDENT_GEN.astype('category')
    df.LOCAL_OCOR = df.LOCAL_OCOR.astype('category')
    df.ID_MN_RESI = df.ID_MN_RESI.astype('category')
    df.CS_RACA = df.CS_RACA.astype('category')
    df.CS_SEXO = df.CS_SEXO.astype('category')

    # remove qualquer linha que possa contar dados nulos ainda
    df.dropna(inplace=True)
    return df

In [30]:
def load_feminicidio() -> pd.DataFrame:
    df = pd.read_csv('../assets/feminicidio_2022.csv', sep=';')
    df = pd.concat([df, pd.read_csv('../assets/feminicidio_2023.csv', sep=';')])
    df = pd.concat([df, pd.read_csv('../assets/feminicidio_2021.csv', sep=';')])

    df.data_fato = pd.to_datetime(df.data_fato, format='%Y-%m-%d')
    # df.tentado_consumado = df.tentado_consumado.astype('category')
    df.qtde_vitimas = df.qtde_vitimas.astype(int)
    df.municipio_fato = df.municipio_fato.apply(lambda x: x.title())

    return df

In [43]:
def load_violencia() -> pd.DataFrame:
    def parse_data(x):
        try:
            return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')
        except:
            try:
                # 45104
                data_base = datetime(1900, 1, 1)
                return data_base + timedelta(days=x -2)
            except:
                return pd.NaT

    df = pd.read_csv('../assets/violencia_domestica_2023.csv', sep=';')

    df.data_fato = df.data_fato.apply(parse_data)
    return df

## Validando alguns dados

In [44]:
df = load_data()
fe = load_feminicidio()
vi = load_violencia()

In [32]:
df.nunique()

DT_NOTIFIC     1095
DT_NASC       23646
NU_IDADE_N      110
CS_SEXO           1
CS_RACA           6
ID_MN_RESI      849
LOCAL_OCOR       10
OUT_VEZES         3
LES_AUTOP         3
VIOL_FISIC        3
VIOL_PSICO        3
VIOL_SEXU         3
NUM_ENVOLV        3
AUTOR_SEXO        4
ORIENT_SEX        5
IDENT_GEN         5
dtype: int64

In [33]:
df.LOCAL_OCOR.value_counts()

LOCAL_OCOR
Residencia                    73681
Ignorado                       9918
Via pública                    8648
Outro                          4182
Bar ou similar                 1665
Escola                         1301
Comercio/Serviços               850
Habitação coletiva              472
Local de pratica esportiva      177
Industria/Construção             74
Name: count, dtype: int64

In [34]:
df[df['ID_MN_RESI']== "Belo Horizonte"]['LOCAL_OCOR'].value_counts().idxmax()

'Residencia'

In [35]:
df[df['ID_MN_RESI']== "Belo Horizonte"]['NU_IDADE_N'].std()

np.float64(20.577093784589277)

In [36]:
df[df['ID_MN_RESI'] == "Verdelândia"][['LES_AUTOP', 'VIOL_FISIC', 'VIOL_PSICO', 'VIOL_SEXU']].groupby(axis=1, level=0).apply(lambda x: x.value_counts().loc['Sim'].sum()).reset_index().columns = ['Tipo de Violência', 'Quantidade']

/tmp/ipykernel_75724/591864412.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df[df['ID_MN_RESI'] == "Verdelândia"][['LES_AUTOP', 'VIOL_FISIC', 'VIOL_PSICO', 'VIOL_SEXU']].groupby(axis=1, level=0).apply(lambda x: x.value_counts().loc['Sim'].sum()).reset_index().columns = ['Tipo de Violência', 'Quantidade']


In [37]:
from unidecode import unidecode
fe[fe.municipio_fato == unidecode("Belo Horizonte")].groupby(['tentado_consumado']).size().reset_index()

,tentado_consumado,0
0,CONSUMADO,38
1,TENTADO,60


In [51]:
vi.tentado_consumado.value_counts()

tentado_consumado
CONSUMADO    60951
TENTADO        585
Name: count, dtype: int64